ici $p_{i,j}$ s'encode `p[j,i]`

In [2]:
import numpy
from matplotlib import pyplot
import matplotlib.cm as cm
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
%matplotlib inline

In [3]:
nx = 41
ny = 41

l = 1.
h = 1.

dx = l/(nx-1)
dy = h/(ny-1)

l1_target = 1e-6

In [4]:
def L1norm(new, old):
    norm = numpy.sum(numpy.abs(new-old))/numpy.sum(numpy.abs(old))
    return norm

In [5]:
def gradient(p,r,d):
    
    Ad=numpy.zeros((ny,nx))  #initialise les tableaux qui stockeront les matrices des gradients
    #Ap=numpy.zeros((ny,nx)) #n'est pas nécessaire
    
    #copie des tableaux reçus pour pouvoir les modifier sans changer ce qui a été recalculé
    pn=p.copy()
    rn=r.copy()
    dn=d.copy()
    
    #mise à jour de la premiere progression de calcul
    rho=numpy.sum(rn*rn)
    Ad[1:-1,1:-1] = -4*dn[1:-1,1:-1]+dn[1:-1,2:]+dn[1:-1,:-2]+\
            dn[2:, 1:-1] + dn[:-2, 1:-1]
    #Ap[1:-1,1:-1] = -4*pn[1:-1,1:-1]+pn[1:-1,2:]+pn[1:-1,:-2]+\
            #pn[2:, 1:-1] + pn[:-2, 1:-1] #n'est pas nécessaire
    sigma=numpy.sum(Ad*dn)
    alpha=rho/sigma

    p=pn+alpha*dn
    
    r=rn-alpha*Ad
   
    beta=numpy.sum(r*r)/rho
    d=r+beta*dn

    return p,r,d

In [9]:
#conditions initiales
pw=numpy.zeros((ny,nx))
pw[-1,:]=-3/dy
pw0=pw.copy()
bw=numpy.zeros((ny,nx))
rw=numpy.zeros((ny,nx))
rw[1:-1,1:-1] = bw[1:-1,1:-1]*dx**2 + 4*pw[1:-1,1:-1] - \
        pw[1:-1,2:] - pw[1:-1,:-2] - pw[2:, 1:-1] - pw[:-2, 1:-1]
dw=rw.copy()

ppsi=numpy.zeros((ny,nx))
ppsi[:,-2]=-dy
ppsi0=ppsi.copy()
bpsi=-pw.copy()
rpsi=numpy.zeros((nx,ny))
rpsi[1:-1,1:-1] = bpsi[1:-1,1:-1]*dx**2 + 4*ppsi[1:-1,1:-1] - \
        ppsi[1:-1,2:] - ppsi[1:-1,:-2] - ppsi[2:, 1:-1] - ppsi[:-2, 1:-1]
dpsi=rpsi.copy()

L1=1+l1_target
l1_w=1.+l1_target
l1_psi=1.+l1_target

i=0

#while L1>l1_target:  # changer critère d'arret
    #print("coucou")
    while l1_psi>l1_target:#calcul de psi

        ppsin=gradient(ppsi.copy(),rpsi.copy(),dpsi.copy())[0].copy()
        rpsin=gradient(ppsi.copy(),rpsi.copy(),dpsi.copy())[1].copy()
        dpsin=gradient(ppsi.copy(),rpsi.copy(),dpsi.copy())[2].copy()
        
        #conditions de bord sur psi (dirichlet et neumann)
        ppsin[0,:]=0 
        ppsin[-1,:]=0
        ppsin[:,0]=0
        ppsin[:,-1]=0
        
        ppsin[1,:]=ppsin[0,:]
        ppsin[-2,:]=ppsin[-2,:]-dy
        ppsin[:,1]=ppsin[:,0]
        ppsin[:,-2]=ppsin[:,-1]
        
        l1_psi=L1norm(ppsin,ppsi)
        
        ppsi=ppsin.copy()
        rpsi=rpsin.copy()
        dpsi=dpsin.copy()
    
    while l1_w>l1_target: #calcul de oméga

        pwn=gradient(pw.copy(),rw.copy(),dw.copy())[0].copy()
        rwn=gradient(pw.copy(),rw.copy(),dw.copy())[1].copy()
        dwn=gradient(pw.copy(),rw.copy(),dw.copy())[2].copy()
    
        #conditions de bord sur pwn 
        pwn[-1,:]=1/(2*dy**2)*(ppsi[-3,:]-8*ppsi[-2,:])-3/dy #top
        pwn[0,:]= 0  #bottom
        pwn[:,0]= 0  #left
        pwn[:,-1]= 0  #right
    
        pwn[-2,:]=pwn[-1,:]
        pwn[1,:]=pwn[0,:]
        pwn[:,1]=pwn[:,0]
        pwn[:,-2]=pwn[:,-1]
        
        l1_w=L1norm(pwn,pw)    

        pw=pwn.copy()
        rw=rwn.copy()
        dw=dwn.copy()
    
    rpsi[1:-1,1:-1] = -pw[1:-1,1:-1]*dx**2 + 4*ppsi[1:-1,1:-1] - \
        ppsi[1:-1,2:] - ppsi[1:-1,:-2] - ppsi[2:, 1:-1] - ppsi[:-2, 1:-1]
    dpsi=rpsi.copy()
    
    L1=max(L1norm(pw,pw0),L1norm(ppsi,ppsi0))
    
    i+=1
    if i>1000000:
        break

coucou


en changeant de place les conditions de bord de pwn, seul le premier chiffre change, le dernier chiffre est identique dans les deux cas


    if i==0:
        fig = pyplot.figure(figsize=(8,5))
        pyplot.subplot(121)
        pyplot.imshow(pw[5:-5,5:-5], cmap = cm.RdBu)
        pyplot.xticks([]), pyplot.yticks([]);
        print(l1)
    
    i+=1
    i=i%100

`pwn[-1,:]=1/(2*dy**2)*(ppsin[-3,:]-8*ppsin[-2,:])-3/dy #top`

`pwn[0,:]= 0 #1/(2*dy**2)*(ppsin[2,:]-8*ppsin[1,:]) #bottom`

`pwn[:,0]= 0 #1/(2*dx**2)*(ppsin[:,2]-8*ppsin[:,1]) #left`

`pwn[:,-1]= 0 #1/(2*dx**2)*(ppsin[:,-3]-8*ppsin[:,-2]) #right`